In [ ]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_constraint.jl")
include("./trajopt/scaling.jl")
# using Plots

In [ ]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/nominal_traj_unicycle_0324" my_dict
# @load "./data/nominal_traj_unicycle_N4" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"];
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [ ]:
dynamics = Unicycle()
ix = dynamics.ix
iu = dynamics.iu
alpha = 0.1
# DLMI = LinearDLMI(alpha,ix,iu)
DLMI = LinearFOH(alpha,ix,iu)
# DLMI = LinearQS(alpha,ix,iu)

In [ ]:
function get_H_obs(rx,ry)
    return diagm([1/rx,1/ry])
end
c_list = []
H_list = []
c1 = [1,2]
H1 = get_H_obs(0.5,0.5)
push!(c_list,c1)
push!(H_list,H1)
c2 = [4,3]
H2 = get_H_obs(0.5,0.5)
push!(c_list,c2)
push!(H_list,H2)

vmax = 2.0
vmin = 0.0
wmax = 2.0
wmin = -2.0
list_const = [InputConstraint([1;0],vmax),
    InputConstraint([-1;0],-vmin),
    InputConstraint([0; 1],wmax),
    InputConstraint([0; -1],-wmin),
    ObstacleAvoidance(H_list[1],c_list[1]),
    ObstacleAvoidance(H_list[2],c_list[2])
    ]

In [ ]:
p2 = Plots.plot(; size=(500,500))
plot!(xnom[1,:],xnom[2,:],c=:deepskyblue3,linestyle=:dash,linewidth=1.5,label=nothing)
scatter!(xnom[1,:],xnom[2,:],c=:deepskyblue3,linewidth=1.5,label=nothing)
for (idx,(ce, H)) in enumerate(zip(c_list, H_list))
    label = nothing
    if idx == 1
        label = "obstacle"
    end
    plot_ellipse(p2,inv(H)*inv(H),ce,"red3",label=label)
end
annotate!(0.2, 0.5, text("Initial", :black, :center))
annotate!(4.8, 4.5, text("Final", :black, :center))
display(p2)
# savefig("./data_image/nominal_trajectory_unicycle.pdf")


In [ ]:
# dynamics.β .= [vmax,1.0,vmax,1.0]
dynamics.β .= [2.0,2.0]
# dynamics.β .= [2*sqrt(2)]
println(dynamics.β)
θ0 = 1.0

In [ ]:
Qnom = zeros(ix,ix,N+1)
Ynom = zeros(iu,ix,N+1)
Snom = zeros(iu,iu,N+1)
Znom = zeros(ix,ix,N+1);
Qini = diagm([0.08,0.08,0.06])
Qf = diagm([0.08,0.08,0.06])

In [ ]:
xmin = [0;0;0];
xmax = [5;5;pi];
umin = [0;0];
umax = [vmax;wmax];
scaler = Scaling(xmin, xmax, umin, umax, tnom[end],0,0)

In [ ]:
include("./funlopt/funl_synthesis.jl")

In [ ]:
max_iter = 1;
w_funl = 1.0;
w_vc = 1e2;
w_tr::Float64 = 0.00
tol_vc = 1e-6;
tol_tr = 1e-5;
tol_dyn = 1e-1;
verbosity = true;

In [ ]:
fs = FunnelSynthesis(N,max_iter,dynamics,DLMI,list_const,scaler,
    w_funl,w_vc,w_tr,tol_tr,tol_vc,tol_dyn,verbosity,
    flag_type="Lsmooth")

In [ ]:
run(fs,Qnom,Ynom,Znom,Qini,Qf,xnom,unom,dtnom,"Mosek",θ0)
# run(fs,Qnom,Ynom,Znom,Qini,Qf,xnom,unom,dtnom,"Clarabel",θ0)

In [ ]:
tprop,xprop,uprop = fs.solution.tprop,fs.solution.xprop,fs.solution.uprop
Qprop,Yprop,Zprop = fs.solution.Qprop,fs.solution.Yprop,fs.solution.Zprop

In [ ]:
using Interpolations
fit_b = LinearInterpolation(tnom, fs.solution.b,extrapolation_bc=Flat());
bprop = fit_b(tprop);

In [ ]:
radius_Q,angle_Q = get_radius_angle_Ellipse2D(fs.solution.Q)
radius_Qi,angle_Qi = get_radius_angle_Ellipse2D(fs.solution.Qi)
radius_Qf,angle_Qf = get_radius_angle_Ellipse2D(fs.solution.Qf)

In [ ]:
# using Plots
# function plot_ellipse(plot,Q::Matrix,xbar::Vector,color;label=nothing)
#     θ = range(0, 2pi + 0.05; step = 0.05)
#     x_y = √Q[1:2,1:2] * hcat(cos.(θ), sin.(θ))' .+ xbar[1:2]
#     plot!(plot, x_y[1, :], x_y[2, :], c = color,linewidth=2,label=label)
#     plot!(plot, x_y[1, :], x_y[2, :], label = nothing,fill=true, fillcolor=color,alpha=0.3)
#     plot!(legendfontsize=12)
# end

In [ ]:
p2 = Plots.plot(; size=(500,500))
plot!(xnom[1,:],xnom[2,:],c=:deepskyblue3,linestyle=:dash,linewidth=1.5,label=nothing)
for idx in 1:N
    label = nothing
    if idx == 1
        label = "funnel"
    end
    plot_ellipse(p2,fs.solution.Q[:,:,idx],xnom[:,idx],"deepskyblue3",label=label)
end
plot_ellipse(p2,fs.solution.Qi,xnom[:,1],"green3",label="initial and final")
plot_ellipse(p2,fs.solution.Qf,xnom[:,end],"green3")
for (idx,(ce, H)) in enumerate(zip(c_list, H_list))
    label = nothing
    if idx == 1
        label = "obstacle"
    end
    plot_ellipse(p2,inv(H)*inv(H),ce,"red3",label=label)
end
display(p2)
savefig("./data_image/funnel_unicycle.pdf")

# estimate β

In [ ]:
using Distributions, Random

In [ ]:
J_estimation = zeros(fs.dynamics.iψ,N+1)
for idx in 1:N+1
    num_sample = 100
    J_sample = zeros(fs.dynamics.iψ,num_sample)
    for j in 1:num_sample
        # sample
        sqrt_Q = sqrt(fs.solution.Q[:,:,idx])
        K = fs.solution.Y[:,:,idx] * inv(fs.solution.Q[:,:,idx])
        z = randn(ix)
        z .= z / norm(z)
        η = sqrt_Q * z
        x_ = xnom[:,idx] + η
        ξ = K * η
        u_ = unom[:,idx] + ξ
        A,B = diff(fs.dynamics,xnom[:,idx],unom[:,idx])
        f = forward(fs.dynamics,xnom[:,idx],unom[:,idx])
        fnext = forward(fs.dynamics,x_,u_)
        eta_dot = fnext - f
        LHS = eta_dot - A*η - B*ξ
        mu = fs.dynamics.Cμ * η + fs.dynamics.Dμu * ξ

        model = Model(Mosek.Optimizer)
        set_optimizer_attribute(model, "MSK_IPAR_LOG", 0) # Turn off verbosity for Mosek

        # @variable(model,J[1:fs.dynamics.iψ])
        # @variable(model,norm_J)
        # @constraint(model, [norm_J; J] in SecondOrderCone())
        # @constraint(model, LHS == fs.dynamics.G * diagm(J) * mu)

        @variable(model,J[1:fs.dynamics.iψ,1:fs.dynamics.iμ])
        @variable(model,norm_J)
        @constraint(model, [norm_J; vec(J)] in SecondOrderCone())
        @constraint(model, LHS == fs.dynamics.G * J * mu)

        @objective(model,Min,norm_J)
        optimize!(model)
        for i in 1:fs.dynamics.iψ
            J_sample[i,j] = norm(value.(J[i,:]))
        end
        # println(eta_dot)
        # println(LHS)
        # println(mu)
        # println(value.(J))
    end
    J_estimation[:,idx] .= maximum(J_sample,dims = 2)
end

In [ ]:
J_result = zeros(fs.dynamics.iψ,N+1)
for idx in 1:N+1
    J_result[:,idx] .= fs.dynamics.β .* sqrt.(fs.solution.b[idx])
end

In [ ]:
p1 = plot(J_result[1,:])
plot!(J_estimation[1,:])
p2 = plot(J_result[2,:])
plot!(J_estimation[2,:])
plot(p1,p2,layout=(1,2))

In [ ]:
# function get_β_unicycle(x,u)
#     v1 = x[3]
#     v2 = u[1]
#     beta = zeros(4)
#     beta[1] = sqrt((v2*cos(v1))^2 + sin(v1)^2)
#     beta[2] = sqrt(sin(v1)^2)
#     beta[3] = sqrt((v2*sin(v1))^2 + cos(v1)^2)
#     beta[4] = sqrt(cos(v1)^2)
#     return beta
# end
function get_β_unicycle(x,u)
    v1 = x[3]
    v2 = u[1]
    beta = zeros(2)
    beta[1] = sqrt((v2*cos(v1))^2 + 2*sin(v1)^2)
    beta[2] = sqrt((v2*sin(v1))^2 + 2*cos(v1)^2)
    return beta
end


In [ ]:
idx = 1
num_sample = 100
β_estimation = zeros(fs.dynamics.iψ,N+1)
v_estimation = zeros(N+1)
for idx in 1:N+1
    β_sample = zeros(fs.dynamics.iψ,num_sample)
    v_sample = zeros(num_sample)
    for j in 1:num_sample
        sqrt_Q = sqrt(fs.solution.Q[:,:,idx])
        K = fs.solution.Y[:,:,idx] * inv(fs.solution.Q[:,:,idx])
        z = randn(ix)
        z .= z / norm(z)
        η = sqrt_Q * z
        x_ = xnom[:,idx] + η
        ξ = K * η
        u_ = unom[:,idx] + ξ
        v = [x_[3];u_[1]]
        vbar = [xnom[3,idx];unom[1,idx]]
        v_sample[j] = norm(v-vbar)
        β_sample[:,j] .= get_β_unicycle(x_,u_)
    end
    β_estimation[:,idx] .= maximum(β_sample,dims = 2)
    v_estimation[idx] = maximum(v_sample)
end

In [ ]:
β_estimation

In [ ]:
maximum(β_estimation,dims=2)

In [ ]:
maximum(abs.(sqrt.(fs.solution.b) .- v_estimation))

## Check eigenvalue of Q

In [ ]:
min_lam_qprop = zeros(size(Qprop,3))
for i in 1:size(Qprop,3)
    eigvals = eigen(Qprop[:,:,i]).values
    min_lam_qprop[i] = eigvals[1]
end
min_lam_q = zeros(size(fs.solution.Q,3))
for i in 1:size(fs.solution.Q,3)
    eigvals = eigen(fs.solution.Q[:,:,i]).values
    min_lam_q[i] = eigvals[1]
end

In [ ]:
scatter(tnom,min_lam_q,seriestype="o")
plot!(tprop,min_lam_qprop)
plot!(tnom,tnom .* 0,linestyle=:dash)

In [ ]:
findfirst(x -> x < 0, min_lam_q)

## Check eigenvalue of Block DLMI

In [ ]:
θ = θ0[1]
fs.dynamics.β
iψ = fs.dynamics.iψ
iμ = fs.dynamics.iμ

In [ ]:
# function get_H(fs,Qprop,Yprop,Zprop,bprop,θ)
#     N11 =  I(fs.dynamics.iμ) .* (θ ./ ( fs.dynamics.β .* fs.dynamics.β))
#     N22 =  bprop .* θ .* I(fs.dynamics.iψ)
#     LMI11 = -Zprop
#     LMI21 = N22 * fs.dynamics.G'
#     LMI22 = -N22
#     LMI31 = fs.dynamics.Cμ * Qprop + fs.dynamics.Dμu * Yprop
#     LMI32 = zeros(fs.dynamics.iμ,fs.dynamics.iψ)
#     LMI33 = -N11
#     return [LMI11 LMI21' LMI31';
#         LMI21 LMI22 LMI32';
#         LMI31 LMI32 LMI33
#     ]
# end
function get_H_(fs,Qprop,Yprop,Zprop,bprop,xprop,uprop)
    θ = θ0
    Aprop,Bprop = diff(fs.dynamics,xprop,uprop)
    N11 =  I(fs.dynamics.iμ) .* (θ ./ ( fs.dynamics.β .* fs.dynamics.β))
    N22 =  bprop .* θ .* I(fs.dynamics.iψ)
    LMI11 = Aprop*Qprop + Qprop*Aprop' + Bprop*Yprop + Yprop'*Bprop' + fs.funl_dynamics.alpha * Qprop - Zprop
    LMI21 = N22 * fs.dynamics.G'
    LMI22 = -N22
    LMI31 = fs.dynamics.Cμ * Qprop + fs.dynamics.Dμu * Yprop
    LMI32 = zeros(fs.dynamics.iμ,fs.dynamics.iψ)
    LMI33 = -N11
    return [LMI11 LMI21' LMI31';
        LMI21 LMI22 LMI32';
        LMI31 LMI32 LMI33
    ]
end
Hprop = zeros(ix+iψ+iμ,ix+iψ+iμ,length(tprop))
H = zeros(ix+iψ+iμ,ix+iψ+iμ,length(tnom))
for i in 1:length(tprop)
    Hprop[:,:,i] .= get_H_(fs,Qprop[:,:,i],Yprop[:,:,i],Zprop[:,:,i],bprop[i],xprop[:,i],uprop[:,i])
end
for i in 1:length(tnom)
    H[:,:,i] .= get_H_(fs,fs.solution.Q[:,:,i],fs.solution.Y[:,:,i],fs.solution.Z[:,:,i],fs.solution.b[i],xnom[:,i],unom[:,i])
end

In [ ]:
max_lam_Hprop = zeros(size(Hprop,3))
for i in 1:size(Qprop,3)
    eig_vals = eigen(Hprop[:,:,i]).values
    if i == 1
        println(eig_vals)
    end
    max_lam_Hprop[i] = eig_vals[end]
end
max_lam_H = zeros(size(H,3))
for i in 1:size(H,3)
    eig_vals = eigen(H[:,:,i]).values
    if i == 1
        println(eig_vals)
    end
    max_lam_H[i] = eig_vals[end]
end

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(tprop,max_lam_Hprop,"-",color="tab:blue")
plt.plot(tnom,max_lam_H,"o",color="tab:blue")
plt.plot(tprop,tprop*0,"--",color="tab:red")
gcf()

In [ ]:
plt.figure()
plt.plot(tprop,Zprop[1,1,:])
plt.plot(tnom,fs.solution.Z[1,1,:],"o")
gcf()

In [ ]:
function project_onto_input(Q,Y) 
    R = []
    for i in 1:size(Q,3)
        K = Y[:,:,i] * inv(Q[:,:,i])
        push!(R,K*Q[:,:,i]*K')
    end
    projected_input_funl = []
    for j in 1:iu
        a = zeros(iu)
        a[j] = 1
        each_funl = []
        for i in 1:length(R)
            push!(each_funl,sqrt(a'*R[i]*a))
        end
        push!(projected_input_funl,each_funl)
    end
    return projected_input_funl
end

In [ ]:
input_proj_funl_nom = project_onto_input(fs.solution.Q,fs.solution.Y)
input_proj_funl_prop = project_onto_input(Qprop,Yprop)

In [ ]:
plt.figure(figsize=(10,3))
plt.subplot(121)
plt.plot(tnom,unom[1,:],"--",color="black")
plt.plot(tnom,unom[1,:]+input_proj_funl_nom[1],"o",color="tab:blue")
plt.plot(tnom,unom[1,:]-input_proj_funl_nom[1],"o",color="tab:blue")
plt.plot(tprop,uprop[1,:]+input_proj_funl_prop[1],"-",color="tab:blue")
plt.plot(tprop,uprop[1,:]-input_proj_funl_prop[1],"-",color="tab:blue")
plt.plot(tnom,tnom*0 .+ vmax,"--",color="tab:red")
plt.plot(tnom,tnom*0 .+ vmin,"--",color="tab:red")
# plt.ylim([-0.1,2.1])
plt.grid(true)
plt.subplot(122)
plt.plot(tnom,unom[2,:],"--",color="black")
plt.plot(tnom,unom[2,:]+input_proj_funl_nom[2],"o",color="tab:blue")
plt.plot(tnom,unom[2,:]-input_proj_funl_nom[2],"o",color="tab:blue")
plt.plot(tprop,uprop[2,:]+input_proj_funl_prop[2],"-",color="tab:blue")
plt.plot(tprop,uprop[2,:]-input_proj_funl_prop[2],"-",color="tab:blue")
plt.plot(tnom,tnom*0 .+ wmax,"--",color="tab:red")
plt.plot(tnom,tnom*0 .+ wmin,"--",color="tab:red")
# plt.ylim([-2.5,2.5])
plt.grid(true)
gcf()

In [ ]:
# my_dict = Dict("x" => xnom, "u" => unom, "t" => tnom,
#      "Q" => fs.solution.Q, "Y" => fs.solution.Y, "Z" => fs.solution.Z)
# using JLD2, FileIO

# @save "./data/unicycle_ICV_0408" my_dict

# Get samples

In [ ]:
using Random
using LinearAlgebra

In [ ]:
xs_list = []
for i in 1:100
    z = randn(ix)
    z = z / norm(z)
    push!(xs_list,xnom[:,1] + sqrt(fs.solution.Q[:,:,1]) * z)
end

In [ ]:
xsam_fwd,tsam,xsam,usam = [],[],[],[]
for xs in xs_list
    xf_,ts_,xsam_,usam_ = propagate_from_funnel_entry(xs,DLMI,dynamics,xnom,unom,dtnom,
        fs.solution.Q,fs.solution.Y,fs.solution.Z)
    push!(xsam_fwd,xf_)
    push!(tsam,ts_)
    push!(xsam,xsam_)
    push!(usam,usam_)
end

In [ ]:
p2 = Plots.plot(; size=(500,500))
plot!(xnom[1,:],xnom[2,:],c=:deepskyblue3,linestyle=:dash,linewidth=1.5,label=nothing)
for idx in 1:N
    label = nothing
    if idx == 1
        label = "funnel"
    end
    plot_ellipse(p2,fs.solution.Q[:,:,idx],xnom[:,idx],"deepskyblue3",label=label)
end
plot_ellipse(p2,fs.solution.Qi,xnom[:,1],"green3",label="initial and final")
plot_ellipse(p2,fs.solution.Qf,xnom[:,end],"green3")
for (idx,(ce, H)) in enumerate(zip(c_list, H_list))
    label = nothing
    if idx == 1
        label = "obstacle"
    end
    plot_ellipse(p2,inv(H)*inv(H),ce,"red3",label=label)
end
for x_ in xsam
    plot!(x_[1,:],x_[2,:],color="purple",label=nothing)
end
display(p2)
savefig("./data_image/funnel_unicycle_samples.pdf")